In [ ]:
%pylab inline

In [ ]:
import numpy
import pandas

In [ ]:
hist_kw = dict(bins=60, normed=True, alpha=0.5)

# KS investigation (with negative weights)

Download 

* `check_agreement.csv`, 

to the folder `datasets/` from https://www.kaggle.com/c/flavours-of-physics/data

It is a control channel, where 1-label corresponds to MC, 0-label - real data, `weight` - sPlot weights

In [ ]:
data_agreement = pandas.read_csv('datasets/check_agreement.csv')
features_for_ks = ['LifeTime', 'FlightDistance', 'IPSig', 'SPDhits']

In [ ]:
for feature in features_for_ks:
    hist(data_agreement[data_agreement.signal == 1][feature].values, label='MC', **hist_kw)
    hist(data_agreement[data_agreement.signal == 0][feature].values, label='real', **hist_kw)
    legend()
    title(feature)
    show()

### Check how sPlot weights change real data pdfs

In [ ]:
# Plot the same histograms with sPlot weights

## Write KS for weights pdfs
standard function doesn't support weights :(

In [ ]:
from sklearn.metrics import roc_curve
def compute_ks(pdf1, pdf2, weights1=None, weights2=None):
    # Write KS metric between two weighted pdfs (see slides)
    ...

## KS metric pdf generation

There are several things to know before doing it:

* we cannot produce two samples from the same distribution with the weights, thus if we generate two samples from uniform and then compute weighted ks metric, obtained value will be greater than it is. That is why during testing hypothesis the probability of error type I will be greater than it is really (Our KS pdf will give the upper estimation of probability to reject H0, when it is true)

* the second sample contains signal and background, which are sampled from different distributions.  
In the second sample we need to generate not only signal (from the same distribution), but also bck from any distribution. But if we remember about sPlot weights property (they compensate bck) then we can conclude that generated bck could be compensated in sample using weights. Of course, some fluctuations can arise (maybe we should somehow estimate them)!

* in the first approximation we can generate two samples as before

###TODO

* Check that for unweighted samples KS will be lower than for weighted.

In [ ]:
# finish the function
def generate_ks_pdf(n1, n2, w1=None, w2=None, points=3000):
    ks = []
    # for each point 
    for step in range(points):
        # generate pdf1 and pdf2 from the same distributions
        ...
        # compute ks for generated pdfs
        ...
    return numpy.array(ks)

In [ ]:
n1 = numpy.sum(data_agreement.signal == 0)
n2 = numpy.sum(data_agreement.signal == 1)
w1 = data_agreement[data_agreement.signal == 0]['weight'].values
w2 = data_agreement[data_agreement.signal == 1]['weight'].values
ks_pdf = generate_ks_pdf(n1, n2)
ks_pdf_weight = generate_ks_pdf(n1, n2, w1=w1, w2=w2)

In [ ]:
hist(ks_pdf, label='simple', **hist_kw)
hist(ks_pdf_weight, label='weight', **hist_kw)
legend()
pass

###TODO:
What features have least disagreement among `features_for_ks`?

# Agreement restriction on the classifiers

How more disagreement features can influence the predictions agreement and classifier's quality?

Do for this:

* train any ensemble model on all features
* remove `SPDhits` from training

In the competition your ks metric should be less than **0.09**

In [ ]:
data = pandas.read_csv('datasets/training.csv')
train_features = list(set(data_agreement.columns) - {'id', 'signal', 'weight'})
train_features_wo_spd = list(set(data_agreement.columns) - {'id', 'signal', 'weight', 'SPDhits'})

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [ ]:
# Divide train on train, test
train_index, test_index = train_test_split(range(len(data)))
train = data.iloc[train_index, :]
test = data.iloc[test_index, :]

In [ ]:
# define function to test model on ks and calculate quality
def test_model(model, features):
    probs = model.predict_proba(data_agreement[features])[:, 1]
    pdf1 = probs[data_agreement.signal.values == 0]
    pdf2 = probs[data_agreement.signal.values == 1]
    model_agr = compute_ks(pdf1, pdf2, weights1=w1, weights2=w2)
    print 'Agreement', model_agr, model_agr < 0.09
    print 'AUC', roc_auc_score(test.signal.values, model.predict_proba(test[features])[:, 1])

### Does subsample parameter influence the agreement and the quality?

### Does learning rate influence?

# Example of overfitting-complexity

In [ ]:
gb = GradientBoostingClassifier(n_estimators=300, learning_rate=0.7, min_samples_leaf=50, subsample=0.5, 
                                max_features=20, max_depth=6)
gb.fit(train[train_features_wo_spd], train['signal'].values)

steps = []
for probs in gb.staged_predict_proba(test[train_features_wo_spd]):
    steps.append(roc_auc_score(test.signal.values, probs[:, 1]))
steps_train = []
for probs in gb.staged_predict_proba(train[train_features_wo_spd]):
    steps_train.append(roc_auc_score(train.signal.values, probs[:, 1]))

In [ ]:
figsize(8, 6)
plot(range(len(steps)), steps, label='test')
plot(range(len(steps_train)), steps_train, label='train')
legend(loc='best')
ylabel('AUC')
xlabel('n_esimators')

The quality on the train is increasing while the quality on the test is decreasing. This is really overfitting. We need take those number of iterations for which quality on test sample is qrowing.

### Summarize:

* The simple solution to improve an agreement is to remove too disagreement features from the training. 
* There are exist smart solutions which will be study on the next seminars.

# Training on sPlot data
There are several ways how to train classifier on sPlot data:

* Choose some threshold > 0 on weights and suppose that events with greater weights are signal and threshold < 0 to define background (this is in consistence with weights meaning).
* As weights are linear combination of $p_s(y)$ and $p_b(y)$, one can put all events with $p_s(y)$ weight as signal and all events with $1 - p_s(y)$ as background.
* apply bagging and select signal event from all events with $p_s(y)$ probability and backgroud events with $p_b(y)$ one.
* apply bagging and and put 1-label with $p_s(y)$ probability and 0-labelwith $p_b(y)$.
* maybe other ways

**TODO**: 

* Try all listed ways compared them on the test dataset (`test_splot`). Test contains MC for $\tau\to 3\mu$ and its sideband. On this data quality will be lower than it is actually is, because $\tau\to 3\mu$ differs from $D_s\to\phi\pi$
* Can you suggest another way how to train classifier on sPlot data?

**Note: the $p_s$ column is defined in the file `probabilities.csv` in folder `datasets`**

In [ ]:
read_columns = list(set(data_agreement.columns) - {'weight'})

In [ ]:
splot_data = data_agreement[data_agreement.signal == 0]
# probabilities to be signal events
splot_probs = pandas.read_csv('datasets/probabilities.csv')['probs'].values
test_splot = data

In [ ]:
# plot weights
hist(splot_data.weight.values, **hist_kw)
pass

### Choose weight thresholds (say >1. for signal and  <-0.4 for bck)
Remember that selected parts of data are not actually signal and bck.

### Add events two times in training

First time as signal with weight = probability_signal.

Second time as background with weight = 1 - probability_signal. 

This method is equivalent to the following function optimization in logloss case for boosting:

$\mathbb{L} = \sum_{y} p_s(y) * \log{\hat{p}(y)} + (1 - p_s(y)) * \log{(1 - \hat{p}(y))} =  \sum_{y} label(y) * w(y) \log{\hat{p}(y)} +  (1 - label(y)) *  w(y)  \log{(1 - \hat{p}(y))}$, 

where $\hat{p}$ is classifier's prediction and the right side is logloss in boosting

### Lirical thoughts: How this result can help to search for $\tau\to 3\mu$? 

In [ ]:
train['new_feature'] = your_model.predict_proba(train[train_features])[:, 1]
test['new_feature'] = your_model.predict_proba(test[train_features])[:, 1]
data_agreement['new_feature'] = your_model.predict_proba(data_agreement[train_features])[:, 1]

In [ ]:
gb_new = GradientBoostingClassifier(n_estimators=300, learning_rate=0.05, min_samples_leaf=50, subsample=0.7, 
                                    max_features=20, max_depth=6)
gb_new.fit(train[train_features_wo_spd + ['new_feature']], train['signal'].values)
test_model(gb_new, train_features_wo_spd + ['new_feature'])

### Bagging with events sampler

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn import clone
# define sampler corresponding to probabilities (generate signal events using ifs pdf, the same for bck)
class SampleSampler(BaseEstimator, ClassifierMixin):
    def __init__(self, base_estimator, n_estimators=10, bootstrap=False, subsample=0.5):
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.bootstrap = bootstrap
        self.subsample = subsample
        
    def fit(self, X, y, sample_weight=None):
        # finish method: generate samples
        if sample_weight is None:
            sample_weight = numpy.ones(len(y))
        y = numpy.array(y)
        X = numpy.array(X)
        
        sig_size = ...
        p_sig = sample_weight[y == 1]
        p_sig /= p_sig.sum()
        
        bck_size = ...
        p_bck = 1 - sample_weight[y == 0]
        p_bck /= p_bck.sum()
        self.estimators = []
        for _ in range(self.n_estimators):
            base = clone(self.base_estimator)
            used_samples = ...
            base.fit(X[used_samples, :], y[used_samples], sample_weight=sample_weight[used_samples])
            self.estimators.append(base)
        return self
    
    def predict_proba(self, X):
        result = numpy.zeros([len(X), 2])
        for clf in self.estimators:
            result += clf.predict_proba(X)
        return result / len(self.estimators)
    
    def predict(self, X):
        return numpy.argmax(self.predict_proba(X), axis=1)
            

In [ ]:
# Train SampleSampler


### Another sampler

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn import clone
# define sampler corresponding to probabilities (generate labels for event using probabilities)
class BaggingSampler(BaseEstimator, ClassifierMixin):
    def __init__(self, base_estimator, n_estimators=10):
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        
    def fit(self, X, y, sample_weight=None):
        # finish method
        if sample_weight is None:
            sample_weight = numpy.ones(len(y))
        y = numpy.array(y)
        X = numpy.array(X)
        # put for event label 1 with p_s probability and label 0 otherwise (sample_weight is p_s) 
        new_y = ...
        self.estimators = []
        for _ in range(self.n_estimators):
            ...
        return self
    
    def predict_proba(self, X):
        result = numpy.zeros([len(X), 2])
        for clf in self.estimators:
            result += clf.predict_proba(X)
        return result / len(self.estimators)
    
    def predict(self, X):
        return numpy.argmax(self.predict_proba(X), axis=1)
            

In [ ]:
# train BaggingSampler


## Summarize:

* Different ways are possible to train classifier using probablity information from data.
* Sometimes you need to train on sPlot data (take only singal) vs MC background. In this case you have two possibilities how to choose signal events
    * make cut on sPlot weights (note, that only classifiers from TMVA library can use negative weights for events)
    * choose events using sampling with signal probability